In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
import tushare as ts
tsStock = TsStock();
chart = Chart();
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
ts.set_token(token)
pro=ts.pro_api()


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [44]:
stock = '300059'
start_date = '20080101'
end_date=''

In [45]:
stock_info = tsStock.get_stock_info(stock)
df_income = pro.income(ts_code=stock_info['code'], start_date=start_date, end_date=end_date)
#df_income.to_excel("income.xlsx",sheet_name='Sheet_name_1')
df_income.index = pd.to_datetime( df_income.ann_date )
df_income = df_income.sort_index(ascending=True)

In [46]:
c = chart.get_line_income(df_income,stock_info)
c.render_notebook()

In [14]:
df_income[['n_income_attr_p']]


,n_income_attr_p
ann_date,
2015-11-13,1.532576e+07
2015-11-13,1.759309e+07
2017-01-11,2.259865e+07
2017-01-11,2.479022e+07
2017-03-06,3.666519e+07
2017-03-06,5.070192e+07
2017-03-06,6.977855e+07
2017-04-28,8.255217e+06
2017-04-28,1.035447e+07


In [15]:
df_income

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,minority_gain,oth_compr_income,t_compr_income,compr_inc_attr_p,compr_inc_attr_m_s,ebit,ebitda,insurance_exp,undist_profit,distable_profit
ann_date,,,,,,,,,,,,,,,,,,,,,
2015-11-13,300630.SZ,20151113,20151113,20121231,1,1,0.1800,0.1800,1.111806e+08,1.111806e+08,...,None,None,1.532576e+07,1.532576e+07,None,2.048582e+07,2.868952e+07,None,None,None
2015-11-13,300630.SZ,20151113,20151113,20150630,1,1,0.1900,0.1900,9.192050e+07,9.192050e+07,...,None,None,1.759309e+07,1.759309e+07,None,1.822564e+07,2.463934e+07,None,None,None
2017-01-11,300630.SZ,20170111,20170111,20160630,1,1,0.2500,0.2500,1.031247e+08,1.031247e+08,...,None,None,2.259865e+07,2.259865e+07,None,2.320437e+07,2.917187e+07,None,None,None
2017-01-11,300630.SZ,20170111,20170111,20131231,1,1,0.2700,0.2700,1.324567e+08,1.324567e+08,...,None,None,2.479022e+07,2.479022e+07,None,2.734368e+07,3.745418e+07,None,None,None
2017-03-06,300630.SZ,20170306,20170306,20141231,1,1,0.4000,0.4000,1.645622e+08,1.645622e+08,...,None,None,3.666519e+07,3.666519e+07,None,3.594157e+07,4.759183e+07,None,None,None
2017-03-06,300630.SZ,20170306,20170306,20151231,1,1,0.5500,0.5500,2.028228e+08,2.028228e+08,...,None,None,5.070192e+07,5.070192e+07,None,5.098037e+07,6.391788e+07,None,None,None
2017-03-06,300630.SZ,20170306,20170306,20161231,1,1,0.7600,0.7600,2.480092e+08,2.480092e+08,...,None,None,6.977855e+07,6.977855e+07,None,7.138165e+07,8.355452e+07,None,None,None
2017-04-28,300630.SZ,20170428,20170428,20160331,1,1,0.0900,0.0900,3.598481e+07,3.598481e+07,...,None,None,8.255217e+06,8.255217e+06,None,9.825370e+06,NaN,None,None,None
2017-04-28,300630.SZ,20170428,20170429,20170331,1,1,0.1100,0.1100,4.823248e+07,4.823248e+07,...,None,None,1.035447e+07,1.035447e+07,None,1.134032e+07,NaN,None,None,None


In [35]:
df = df_income.drop_duplicates(["ann_date"],keep="last")[['n_income_attr_p']]


In [36]:
df['last_year_n_income_attr_p'] = df['n_income_attr_p'].shift(4)
df['last_n_income_attr_p'] = df['n_income_attr_p'].shift(1)

In [37]:
df['income_tongbi'] =  ( df['n_income_attr_p']- df['last_year_n_income_attr_p'] )/(df['last_year_n_income_attr_p'])

In [38]:
df['income_huanbi'] = (df['n_income_attr_p']- df['last_n_income_attr_p'] )/ df['last_n_income_attr_p']

In [39]:
df.rename(columns={'n_income_attr_p': 'n_income'}, inplace=True)

In [40]:
df.tail(1)

,n_income,last_year_n_income_attr_p,last_n_income_attr_p,income_tongbi,income_huanbi
ann_date,,,,,
2019-08-23,1.200845e+08,70304363.48,70014897.74,0.708066,0.715127
